In [38]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np
import os

In [39]:
df=pd.read_csv('./data/oscars.csv')
df

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10760,2022,2023,95,HONORARY AWARD,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,95,HONORARY AWARD,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,95,HONORARY AWARD,"To Peter Weir, a fearless and consummate filmm...",NaN,True
10763,2022,2023,95,GORDON E. SAWYER AWARD,Iain Neil,NaN,True


In [40]:
df=df.loc[df['year_ceremony'] == 2023]
df

,year_film,year_ceremony,ceremony,category,name,film,winner
10639,2022,2023,95,ACTOR IN A LEADING ROLE,Austin Butler,Elvis,False
10640,2022,2023,95,ACTOR IN A LEADING ROLE,Colin Farrell,The Banshees of Inisherin,False
10641,2022,2023,95,ACTOR IN A LEADING ROLE,Brendan Fraser,The Whale,True
10642,2022,2023,95,ACTOR IN A LEADING ROLE,Paul Mescal,Aftersun,False
10643,2022,2023,95,ACTOR IN A LEADING ROLE,Bill Nighy,Living,False
...,...,...,...,...,...,...,...
10760,2022,2023,95,HONORARY AWARD,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,95,HONORARY AWARD,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,95,HONORARY AWARD,"To Peter Weir, a fearless and consummate filmm...",NaN,True
10763,2022,2023,95,GORDON E. SAWYER AWARD,Iain Neil,NaN,True


In [41]:
df=df.dropna(subset=['film'])
df.loc[:, 'category'] = df['category'].str.lower()
df

,year_film,year_ceremony,ceremony,category,name,film,winner
10639,2022,2023,95,actor in a leading role,Austin Butler,Elvis,False
10640,2022,2023,95,actor in a leading role,Colin Farrell,The Banshees of Inisherin,False
10641,2022,2023,95,actor in a leading role,Brendan Fraser,The Whale,True
10642,2022,2023,95,actor in a leading role,Paul Mescal,Aftersun,False
10643,2022,2023,95,actor in a leading role,Bill Nighy,Living,False
...,...,...,...,...,...,...,...
10755,2022,2023,95,writing (original screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,True
10756,2022,2023,95,writing (original screenplay),Written by Steven Spielberg & Tony Kushner,The Fabelmans,False
10757,2022,2023,95,writing (original screenplay),Written by Todd Field,Tár,False
10758,2022,2023,95,writing (original screenplay),Written by Ruben Östlund,Triangle of Sadness,False


In [42]:
df.loc[:, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' to win the award'
df.loc[df['winner'] == False, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' but did not win'
df

/tmp/ipykernel_4802/3596920203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' to win the award'


,year_film,year_ceremony,ceremony,category,name,film,winner,text
10639,2022,2023,95,actor in a leading role,Austin Butler,Elvis,False,Austin Butler got nominated under the category...
10640,2022,2023,95,actor in a leading role,Colin Farrell,The Banshees of Inisherin,False,Colin Farrell got nominated under the category...
10641,2022,2023,95,actor in a leading role,Brendan Fraser,The Whale,True,Brendan Fraser got nominated under the categor...
10642,2022,2023,95,actor in a leading role,Paul Mescal,Aftersun,False,"Paul Mescal got nominated under the category, ..."
10643,2022,2023,95,actor in a leading role,Bill Nighy,Living,False,"Bill Nighy got nominated under the category, a..."
...,...,...,...,...,...,...,...,...
10755,2022,2023,95,writing (original screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,True,Written by Daniel Kwan & Daniel Scheinert got ...
10756,2022,2023,95,writing (original screenplay),Written by Steven Spielberg & Tony Kushner,The Fabelmans,False,Written by Steven Spielberg & Tony Kushner got...
10757,2022,2023,95,writing (original screenplay),Written by Todd Field,Tár,False,Written by Todd Field got nominated under the ...
10758,2022,2023,95,writing (original screenplay),Written by Ruben Östlund,Triangle of Sadness,False,Written by Ruben Östlund got nominated under t...


In [43]:
client = chromadb.Client()
collection = client.get_or_create_collection("oscars-2023")

In [44]:
docs=df["text"].tolist() 

In [47]:
ids= [str(x) for x in df.index.tolist()]

In [48]:
collection.add(
    documents=docs,
    ids=ids
)

Add of existing embedding ID: 10639
Add of existing embedding ID: 10640
Add of existing embedding ID: 10641
Add of existing embedding ID: 10642
Add of existing embedding ID: 10643
Add of existing embedding ID: 10644
Add of existing embedding ID: 10645
Add of existing embedding ID: 10646
Add of existing embedding ID: 10647
Add of existing embedding ID: 10648
Add of existing embedding ID: 10649
Add of existing embedding ID: 10650
Add of existing embedding ID: 10651
Add of existing embedding ID: 10652
Add of existing embedding ID: 10653
Add of existing embedding ID: 10654
Add of existing embedding ID: 10655
Add of existing embedding ID: 10656
Add of existing embedding ID: 10657
Add of existing embedding ID: 10658
Add of existing embedding ID: 10659
Add of existing embedding ID: 10660
Add of existing embedding ID: 10661
Add of existing embedding ID: 10662
Add of existing embedding ID: 10663
Add of existing embedding ID: 10664
Add of existing embedding ID: 10665
Add of existing embedding ID

In [56]:
results = collection.query(
    query_texts=["RRR"],
    n_results=1
)

In [57]:
print(results['documents'])

[['Music by M.M. Keeravaani; Lyric by Chandrabose got nominated under the category, music (original song), for the film RRR to win the award']]
